# Estimator 
### Last Modified: August 26, 2020

To run, press SHIFT+ENTER on each cell. <br>
To get to next cell, press the down arrow after clicking on a blank spot of the cell. <br>
<b> Likewise, you can click on "Restart & Run All" under the Kernel menu (may need to re-run cells with dropdown boxes)</b><br>
Cells can be toggled on and off by running the cell preceding this message.
## If asked to "Try and re-run cell", then click on a blank spot of the cell until no dropdown box or button is selected and press SHIFT+ENTER

In [ ]:
# Widgets
from IPython.display import display, clear_output, HTML, FileLink
import ipywidgets as widgets
from ipywidgets import Layout

# Imports
import pandas as pd
import numpy as np
import math

import io
import requests

import time
from datetime import date

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Variables
scope = "https://spreadsheets.google.com/feeds"
credentials = ServiceAccountCredentials.from_json_keyfile_name("matcher-272116-8801bce55bcb.json", scope)
gs = gspread.authorize(credentials)

estimator_sheet_url = "https://docs.google.com/spreadsheets/d/1EhbS_vM28BQWW2SVCS1RefWD9tzDrWbYT2MdSDlobVM"

# Print Matcher Sheet URL
print("Go to {0} to view Estimator Sheet".format(estimator_sheet_url))

In [ ]:
# Allows window height to be larger
# Doesn't seem to work in Binder?
HTML('''
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:50000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>''');

In [ ]:
# Toggle Code Button
HTML('''<script>
code_show=true;
function code_toggle() {
    if (code_show){
    $('div.input').hide();
    } else {
    $('div.input').show();
    }
    code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code."></form>
''')

# Update Journey Data

In [ ]:
# Creates current journeys update button and corresponding output
update_current_journeys_button = widgets.Button(description = "Update Current Journeys", layout = Layout(width = "200px"))
update_current_journeys_view_expression_button = widgets.Button(description = "Update Actual View and Expression Values", layout = Layout(width = "300px"))
out_update_current_journeys = widgets.Output()

def update_current_journeys(_) :
    with out_update_current_journeys :
        # Clears printed output
        clear_output()
        
        # Updates current journeys
        print("Updating Current Journeys", end = "\r")
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Opens and clears current journeys sheet
        current_journeys_sheet = gs.open_by_url(estimator_sheet_url).worksheet("Current_Journeys")
        current_journeys_sheet.clear()
        
        # Updates Journey worksheet with cyfe board
        current_journeys_sheet.update("A1", gs.open_by_url(estimator_sheet_url).worksheet("URLs").get("B1")[0][0][1:], raw = False)
        
        # Waits for cell A1 to be the value "journey_id"
        while (current_journeys_sheet.acell("A1").value != "#N/A" and current_journeys_sheet.acell("A1").value != "journey_id") :
            time.sleep(5)
            
        # Copies cells to save journey
        current_journeys_sheet.update("A1:R", current_journeys_sheet.get("A1:R"), raw = False)
        
        # Prints out how long it took to run
        print("Updated Current Journeys Worksheet in {0:.3f} secs to run".format(time.time() - start_time))
        # Prints if cyfe board was empty
        if current_journeys_sheet.acell("A1").value == "#N/A" :
            print("No journeys loaded")
            
        # Clears previous results in Information
        gs.open_by_url(estimator_sheet_url).values_clear("Information!M4:R")
        
def update_view_expression(_) :
    with out_update_current_journeys :
        # Clears printed output
        clear_output()
        
        # Updates views and expressions
        print("Updating Actual View and Expression Values", end = "\r")
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        # Opens current journeys sheet
        current_journeys_sheet = gs.open_by_url(estimator_sheet_url).worksheet("Current_Journeys")
        
        # Makes sure current journeys sheet has values already
        if (current_journeys_sheet.acell("A1").value != "journey_id") :
            print("Update Current Journeys First                                   ")
            return
        
        # Gets views and expressions from cyfe board
        views_expressions = pd.read_csv(io.StringIO(requests.get(gs.open_by_url(estimator_sheet_url).worksheet("URLs").get("B2")[0][0][1:]).content.decode('utf-8')))
        # Gets old current journeys data and removes views, expressions, and journey_id_2
        old_current_journeys = pd.DataFrame(current_journeys_sheet.get_all_records()).drop(["views", "expressions", "journey_id_2"], axis = 1)
        
        # Merges old current journeys with new views and expressions
        old_current_journeys = old_current_journeys.merge(views_expressions, how = "left", left_on = "journey_id", right_on = "journey_id_2").fillna("-")
        old_current_journeys["journey_id_2"] = old_current_journeys["journey_id"]
        
        # Clears current journeys sheet and updates it
        current_journeys_sheet.clear()
        current_journeys_sheet.update([old_current_journeys.columns.values.tolist()] + old_current_journeys.values.tolist(), raw = False)
        
        # Prints out how long it took to run
        print("Updated Current Journeys Worksheet's Actual View and Expression Values in {0:.3f} secs to run".format(time.time() - start_time))
        # Prints if cyfe board was empty
        if current_journeys_sheet.acell("P1").value == "#N/A" :
            print("No journeys loaded")
            
# Sets current journeys update buttons with functions
update_current_journeys_button.on_click(update_current_journeys)
update_current_journeys_view_expression_button.on_click(update_view_expression)

# Displays widgets
widgets.VBox([widgets.HBox([update_current_journeys_button,update_current_journeys_view_expression_button]), out_update_current_journeys])

# Estimate Number of Views and Expressions
#### Searches for similar journeys based on past data for the current journeys.
Similar journeys are journeys that have the exact same week day, exact same starting location, exact same number of men, and exact same number of jobs.<br>
Similar journeys also have a similar start time, similar end time, similar total time, similar total distance, similar start to end distance, similar volume, and similar price.<br>
These last 7 variables are not exact matches, but instead within +/- step percent of the current journey's values.<br>
(i.e. If a journey has a volume of 1.54 m^3, then initially it searches for journeys with volumes between 1.5015 m^3 and 1.5785 m^3 which corresponds to +/- 2.5% of the journey's volume)<br>
If not enough similar journeys are found in this initial step range, then the range is increased until a sufficient number of similar journeys are found.

In [ ]:
# Creates load current and past journeys button and corresponding output
load_current_past_journeys_button = widgets.Button(description = "Load Current and Past Journeys", layout = Layout(width = "250px"))
out_load_current_past_journeys = widgets.Output()

# Initializes current_journeys and past_journeys
current_journeys = None
past_journeys = None

def load_current_past_journeys(_) :
    # Allows current_journeys and past_journeys to be saved outside the function
    global current_journeys, past_journeys
    
    with out_load_current_past_journeys :
        # Clears printed output
        clear_output()
        
        # Gets current journey data
        print("Loading Current Journeys", end = "\r")
        current_journeys = pd.DataFrame(gs.open_by_url(estimator_sheet_url).worksheet("Current_Journeys").get_all_records()).fillna(0.0)
        print("Current Journeys Loaded: {0} Journeys".format(len(current_journeys.index)))
        
        # Gets past journey data
        print("Loading Past Journeys", end = "\r")
        past_journeys = pd.DataFrame(gs.open_by_url(estimator_sheet_url).worksheet("Past_Journeys").get_all_records(head = 3)).fillna(0.0)
        # Adds a month to past_journeys
        if "month" not in past_journeys.columns :
            past_journeys["month"] = [past_journeys.strip().split("/" if "/" in date else "-")[1] for date in past_journeys["date"]]
        # Removes all journeys that were returns
        if "return" in past_journeys.columns :
            past_journeys = past_journeys.where(past_journeys["return"] != 1).dropna()
        print("Past Journeys Loaded: {0} Journeys ranging from {1} to {2}".format(len(past_journeys.index), min(past_journeys["date"]), max(past_journeys["date"])))

# Sets load current and past journeys button with function
load_current_past_journeys_button.on_click(load_current_past_journeys)

# Displays widgets
widgets.VBox([load_current_past_journeys_button, out_load_current_past_journeys])

In [ ]:
# Creates journey info dropdown box
# Does this if current_journeys is loaded or displays "Try to re-run cell" if current_journeys is not loaded
if current_journeys is not None :
    journey_info = widgets.Dropdown(
        options = np.insert(np.array(current_journeys["journey_id"].sort_values().unique(), dtype = object), 0, "Select journey"),
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
else :
    journey_info = widgets.Dropdown(
        options = ["Select journey", "Try to re-run cell"],
        value = "Select journey",
        description = "Journey:",
        style = {"description_width":"initial"})
    
# Creates journey info button and corresponding output
journey_info_button = widgets.Button(description = "Display Journey Info", layout = Layout(width = "160px"))
out_journey_info = widgets.Output()

def format_time(time) :
    # Formats time in HH:MM for all times
    format_time = float(time)
    format_hour = int(format_time * 24)
    format_min = int((format_time * 24 - format_hour) * 60)

    return "{0:02d}:{1:02d}".format(format_hour, format_min)

def string_handling(x, float_int) :
    # Handles string if there are commas and converts string to float or int
    try :
        result = float_int(x)
    except :
        result = float_int(x.replace(",",""))
        
    return result

def display_journey_info(_) :
    with out_journey_info :
        # Clears printed output
        clear_output()
        
        # Makes sure an actual journey is selected
        if journey_info.value != "Select journey" and journey_info.value != "Try to re-run cell" and journey_info.value != None :
            # Gets the journey data
            journey = current_journeys.where(current_journeys["journey_id"] == journey_info.value).dropna(how = "all").iloc[0]
            
            # Prints journey data information
            print(u"Journey ID: {0}\nTP Amount: \xa3{1:.2f}\nNumber of Men: {2}\nNumber of Jobs: {3}\nStart to End Distance: {4:.2f} km\nTotal Distance: {5:.2f} km".format(string_handling(journey["journey_id"], int), 
                                                                                                                                                                            string_handling(journey["price"], float),
                                                                                                                                                                            string_handling(journey["number_of_men"], int),
                                                                                                                                                                            string_handling(journey["number_of_jobs"], int),
                                                                                                                                                                            string_handling(journey["finish_distance"], float), 
                                                                                                                                                                            string_handling(journey["km"], float)))
            print("Start Location: {0}\nStart Time: {1}\nEnd Time: {2}\nTotal Time: {3:.2f} hours\nMax Volume: {4:.2f} m^3\nMax Weight: {5} kg".format(journey["start_location"],
                                                                                                                                                       format_time(journey["start_time"]), 
                                                                                                                                                       format_time(journey["end_time"]),
                                                                                                                                                       string_handling(journey["total_time_day"], float) * 24,
                                                                                                                                                       string_handling(journey["volume"], float), 
                                                                                                                                                       string_handling(journey["kg"], float)))
            # Prints out estimates if calculated
            if estimates is not None :
                # Gets the estimates
                journey_estimate = estimates.where(estimates["journey_id"] == journey_info.value).dropna(how = "all").iloc[0]
                
                # Prints out estimates and actual
                print("\n{0} Similar Journeys within {1:.1f}% of Values\n".format(journey_estimate["number_similar"], journey_estimate["value_range"] * 100))
                print("Estimated Views: {0:.1f}\nActual Views: {1}".format(journey_estimate["pred_views"], journey["views"]))
                print("Estimated Expressions: {0:.1f}\nActual Expressions: {1}\n".format(journey_estimate["pred_expressions"], journey["expressions"]))
                
                # Prints out data_overlap
                print("Similar Journeys:")
                if len(journey_estimate["data_overlap"].index) > 0 :
                    display(journey_estimate["data_overlap"][["job_id", "date", "views", "expressions", "journey_tp_price", "number_of_men", 
                                                              "active_listing_count", "start_end", "km", "start_n1", "start_time", "end_time",
                                                              "total_time_day", "m3", "kg"]].rename(
                                                columns = {"job_id" : "Journey ID", "date" : "Date", "views" : "Views", "expressions" : "Expressions", "journey_tp_price" : u"TP Amount (\xa3)", "number_of_men" : "Number of Men", 
                                                           "active_listing_count" : "Number of Jobs", "start_end" : "Start to End Distance (km)", "km" : "Total Distance (km)", "start_n1" : "Start Location", "start_time" : "Start Time (day)", "end_time" : "End Time (day)",
                                                           "total_time_day" : "Total Time (day)", "m3" : "Max Volume (m^3)", "kg" : "Max Weight (kg)"}))
                else :
                    print("None")
            
# Sets the journey info button with function
journey_info_button.on_click(display_journey_info)

# Displays widgets
widgets.VBox([widgets.HBox([journey_info, journey_info_button]), out_journey_info])

In [ ]:
# Initializes estimates
estimates = None
estimates_columns = ["journey_id", "pred_views", "pred_expressions", "number_similar", "value_range", "data_overlap"]

# Amount to increment step by
step_increment = 0.025

# Creates number of random sorting iterations
min_number_similar = widgets.IntText(
                     value = 2,
                     description = "Minimum number of similar journeys:",
                     style = {"description_width":"initial"})

def estimate_number(journey, step, iteration, number_iterations) :
    # Allows estimates to be saved outside of the function
    global estimates, estimates_columns
    
    data_overlap = data_overlap_prime(journey, step)
    
    count_view = data_overlap["views"].mean()
    count_express = data_overlap["expressions"].mean()
    
    print("Working on journey {0}: {1} of {2} journeys (Step: {3:.1f}%, Number Similar: {4})        ".format(string_handling(journey["journey_id"], int), 
                                                                                                             iteration + 1, 
                                                                                                             number_iterations,
                                                                                                             step * 100, 
                                                                                                             len(data_overlap.index)), end = "\r")
    
    if len(data_overlap.index) < min_number_similar.value and step <= 5 :
        estimate_number(journey, step + step_increment, iteration, number_iterations)
    else :
        estimates = estimates.append(pd.DataFrame([[journey["journey_id"], count_view, count_express, len(data_overlap.index), step, data_overlap]], columns = estimates_columns))
        
def data_overlap_prime(journey, step) :
    data = past_journeys.where(past_journeys["week_day"] == string_handling(journey["week_day"], int)).dropna(how = "all")
    data = data.where(data["start_n1"] == journey["start_location"]).dropna(how = "all")
    data = data.where(data["number_of_men"] == string_handling(journey["number_of_men"], int)).dropna(how = "all")
    data = data.where(data["active_listing_count"] == string_handling(journey["number_of_jobs"], int)).dropna(how = "all")
    
    # All variables with step size looking
    data = data.where(np.logical_and(data["start_time"] >= string_handling(journey["start_time"], float) * (1 - step), 
                                     data["start_time"] <= string_handling(journey["start_time"], float) * (1 + step))).dropna(how = "all")
    data = data.where(np.logical_and(data["end_time"] >= string_handling(journey["end_time"], float) * (1 - step), 
                                     data["end_time"] <= string_handling(journey["end_time"], float) * (1 + step))).dropna(how = "all")
    data = data.where(np.logical_and(data["total_time_day"] >= string_handling(journey["total_time_day"], float) * (1 - step), 
                                     data["total_time_day"] <= string_handling(journey["total_time_day"], float) * (1 + step))).dropna(how = "all")
    data = data.where(np.logical_and(data["km"] >= string_handling(journey["km"], float) * (1 - step), 
                                     data["km"] <= string_handling(journey["km"], float) * (1 + step))).dropna(how = "all")
    data = data.where(np.logical_and(data["start_end"] >= string_handling(journey["finish_distance"], float) * (1 - step), 
                                     data["start_end"] <= string_handling(journey["finish_distance"], float) * (1 + step))).dropna(how = "all")
    data = data.where(np.logical_and(data["m3"] >= string_handling(journey["volume"], float) * (1 - step), 
                                     data["m3"] <= string_handling(journey["volume"], float) * (1 + step))).dropna(how = "all")
    return data.where(np.logical_and(data["journey_tp_price"] >= string_handling(journey["price"], float) * (1 - step), 
                                     data["journey_tp_price"] <= string_handling(journey["price"], float) * (1 + step))).dropna(how = "all")

# Displays widgets
min_number_similar

In [ ]:
# Create estimate view and expression button with corresponding button
estimate_views_expressions_button = widgets.Button(description = "Estimate View and Expression Values", layout = Layout(width = "300px"))
out_estimate_views_expressions = widgets.Output()

def estimate_views_expressions(_) :
    # Allows estimates to be saved outside of the function
    global estimates, estimates_columns
    
    with out_estimate_views_expressions :
        # Clears printed output
        clear_output()
        
        # Makes sure min_number_similar is appropriate
        if min_number_similar.value <= 0 :
            print("Select a valid number of similar journeys")
            return
        
        # Initializes estimates
        estimates = pd.DataFrame(columns = estimates_columns)
        
        # Tells you how long it took to run
        # Initializes start time
        start_time = time.time()
        
        for i,journey in current_journeys.iterrows() :
            estimate_number(journey, step_increment, i, len(current_journeys.index))
            print("Completed journey {0}: {1} of {2} journeys (Step: {3:.1f}%, Number Similar: {4})        ".format(string_handling(journey["journey_id"], int),
                                                                                                                    i + 1, 
                                                                                                                    len(current_journeys.index),
                                                                                                                    estimates["value_range"].iloc[i] * 100,
                                                                                                                    estimates["number_similar"].iloc[i]))
            
        # Prints that estimates is done
        print("\nCompleted {0} journeys in {1:.3f} sec".format(len(current_journeys.index), time.time() - start_time))
        
# Sets the estimate view and expression button with function
estimate_views_expressions_button.on_click(estimate_views_expressions)

# Display widgets
widgets.VBox([estimate_views_expressions_button, out_estimate_views_expressions])

# Save Number of Views and Expressions

# Done